# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

Using TensorFlow backend.


In [3]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [24]:
################################
# helper snippet
################################

m = np.matrix([[1, 2], [3, 4]])
print(m)
(rows, columns) = m.shape
for rowIdx in range(rows):
    s = m[rowIdx]
    total = s.sum()
    print(s.sum())
    print(np.divide(s, total))
    print(s.dtype)
    print(s.flatten())
    
    #for value in m[rowIdx, :]:
    #    print(value)
    #print(m[rowIdx, :])
    



[[1 2]
 [3 4]]
3
[[0.33333333 0.66666667]]
int64
[[1 2]]
7
[[0.42857143 0.57142857]]
int64
[[3 4]]


In [35]:
############################
# helper snippet 
############################
nan_indexes = [797, 1090, 1093, 1094, 1095, 1096, 1097]
for sequence in corpus:
    for nan_index in nan_indexes:
        if nan_index in sequence:
            print(sequence)
            break
    
print(tokenizer.word_index)

[797]
[1090]
[1093]
[1094]
[1095]
[1096]
[1097]
{'duck': 374, 'begged': 1081, 'nearly': 871, 'advise': 752, 'have': 84, 'fear': 402, 'heap': 660, 'beasts': 709, 'drink': 338, 'met': 1087, 'choked': 1079, 'chorus': 565, 'pairs': 1182, 'simply': 1071, 'w': 1170, 'way': 45, 'longed': 688, 'leap': 921, 'politely': 999, 'then': 79, 'odd': 794, 'neatly': 854, 'key': 148, 'gave': 474, 'red': 715, 'shrill': 926, 'swim': 1156, 'growled': 1109, 'english': 228, 'hour': 1041, 'continued': 1020, 'nicely': 934, 'side': 441, 'these': 881, 'country': 643, 'shrinking': 517, 'claws': 856, 'jar': 401, 'stairs': 627, 'bed': 1139, 'hippopotamus': 906, 'i': 6, 'bathing': 891, 'hanging': 670, 'despair': 1054, 'conversation': 944, 'times': 359, 'remember': 219, 'sharply': 471, 'miles': 407, 'fur': 546, 'find': 150, 'australia': 645, 'playing': 761, 'getting': 138, 'dark': 305, 'harm': 907, 'various': 1140, 'sight': 328, 'nothing': 112, 'trying': 187, 'four': 214, 'boots': 789, 'child': 477, 'speech': 567, 'th

In [40]:
#create co-occurrence matrix
#print(corpus[0])
coMatrix = np.zeros((V, V))
for sequence in corpus:
    for idx, termId in enumerate(sequence):
        sl = len(sequence)
        ## select left window
        leftw = sequence[max(idx - window_size_corpus, 0): idx]
        ## select right window
        rightw = sequence[idx + 1: min(idx + window_size_corpus + 1, sl)]
        # update co-occurrence matrix
        neighboors = leftw + rightw
        for neighbor in neighboors:
            coMatrix[termId, neighbor] += 1
        

# matrix normalization
(rows, columns) = coMatrix.shape
for rowIdx in range(rows):
    # ignore first row with 0 entries everywhere
    if (rowIdx > 0):
        row = coMatrix[rowIdx]
        total = row.sum()
        # avoid division by zero in words that have no neighboors
        if total > 0:
            coMatrix[rowIdx] = np.divide(row, total)


[242, 6, 26, 1, 63, 243]


In [28]:
print(coMatrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02363823 0.0364851  ... 0.00051387 0.         0.        ]
 [0.         0.04752343 0.01204819 ... 0.00066934 0.00066934 0.00066934]
 ...
 [0.         0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.         0.         0.125      ... 0.         0.         0.        ]
 [0.         0.         0.25       ... 0.         0.         0.        ]]


In [48]:
#find cosine similarity to Alice, Dinah and Rabbit

#find the word vectors for Alice, Dinah, and Rabbit
aliceIdx = tokenizer.word_index['Alice'.lower()]
dinahIdx = tokenizer.word_index['Dinah'.lower()]
rabbitIdx = tokenizer.word_index['Rabbit'.lower()]

aliceVector = coMatrix[aliceIdx].reshape(1, -1)
dinahVector = coMatrix[dinahIdx].reshape(1, -1)
rabbitVector = coMatrix[rabbitIdx].reshape(1, -1)
print(aliceVector)
print(dinahVector)
print(rabbitVector)

cosAD = cosine_similarity(aliceVector, dinahVector)
print("Cosine similarity between Alice and Dinah: {}".format(cosAD))
cosAR = cosine_similarity(aliceVector, rabbitVector)
print("Cosine similarity between Alice and Rabbit: {}".format(cosAR))
cosDR = cosine_similarity(dinahVector, rabbitVector)
print("Cosine similarity between Dinah and Rabbit: {}".format(cosDR))


[[0.         0.02646503 0.03402647 ... 0.         0.         0.        ]]
[[0.         0.03448276 0.03448276 ... 0.         0.         0.        ]]
[[0.         0.13559322 0.00847458 ... 0.         0.         0.        ]]
Cosine similarity between Alice and Dinah: [[0.39360011]]
Cosine similarity between Alice and Rabbit: [[0.47890931]]
Cosine similarity between Dinah and Rabbit: [[0.29862324]]


In [49]:
#find the closest words to Alice
# create an array containing the cosine similarity values for alice and the rest
nbrs = nn(n_neighbors=6, algorithm='brute', metric='cosine').fit(coMatrix)
distances, indices = nbrs.kneighbors(aliceVector)
print(distances)
print(indices)


[[0.         0.18693993 0.22482059 0.25996004 0.26423806 0.2653115 ]]
[[11  4  7  1  5 41]]


In [ ]:
# validate the computed distances and the indices

From the results we observe the six closest words to 'Alice'.

Word with index 11 corresponds to 'Alice' itself. 

The words for the remaining indexes are:
* 4: she
* 7: it
* 1: the
* 5: a
* 41: herself

Discussion of the drawbacks:

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow


In [242]:
#create CBOW model


In [243]:
#define loss function


In [1]:
#train model


In [ ]:
#prepare data for Skipgram


In [ ]:
#create Skipgram model


In [ ]:
#define loss function for Skipgram


In [ ]:
#train Skipgram model


In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer


In [ ]:
#define loss function for Skipgram + dense


In [ ]:
#train model for Skipgram + dense


In [ ]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings